<a href="https://colab.research.google.com/github/weathon/wild-fire/blob/main/WildFire_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dataset Used
# https://www.kaggle.com/datasets/brsdincer/wildfire-detection-image-data

In [ ]:
!unzip archive.zip

In [4]:
!pip install torchinfo

In [25]:
import torch, torchinfo

class Block(torch.nn.Module):
  def __init__(self, i, o):
    super().__init__()
    self.conv1 = torch.nn.Conv2d(i, o, (3, 3), padding=1)
    self.pro1 = torch.nn.Conv2d(i, o, (1, 1))
    self.conv2 = torch.nn.Conv2d(o, o, (3, 3), 2)
    self.relu = torch.nn.ReLU()


  def forward(self, img):
    x = self.relu(self.conv1(img)) + self.pro1(img)
    return self.relu(self.conv2(x))

torchinfo.summary(Block(3, 16), (1,3,250,250))

Layer (type:depth-idx)                   Output Shape              Param #
Block                                    [1, 16, 124, 124]         --
├─Conv2d: 1-1                            [1, 16, 250, 250]         448
├─Conv2d: 1-2                            [1, 16, 250, 250]         64
├─Conv2d: 1-3                            [1, 16, 124, 124]         2,320
Total params: 2,832
Trainable params: 2,832
Non-trainable params: 0
Total mult-adds (M): 67.67
Input size (MB): 0.75
Forward/backward pass size (MB): 17.97
Params size (MB): 0.01
Estimated Total Size (MB): 18.73

In [34]:
class Model(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.block1 = Block(3, 16)
    self.block2 = Block(16, 32)
    self.block3 = Block(32, 64)
    self.block4 = Block(64, 128)
    self.block5 = Block(128, 256)
    self.f = torch.nn.Flatten()
    self.FC1 = torch.nn.Linear(9216, 1024)
    self.relu1 = torch.nn.ReLU()
    self.FC2 = torch.nn.Linear(1024, 16)
    self.relu2 = torch.nn.ReLU()
    self.FC3 = torch.nn.Linear(16, 1)

  def forward(self, x):
    x =  self.f(self.block5(self.block4(self.block3(self.block2(self.block1(x))))))
    return self.FC3(self.relu2(self.FC2(self.relu1(self.FC1(x)))))


torchinfo.summary(Model(), (1024,3,250,250))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1024, 1]                 --
├─Block: 1-1                             [1024, 16, 124, 124]      --
│    └─Conv2d: 2-1                       [1024, 16, 250, 250]      448
│    └─Conv2d: 2-2                       [1024, 16, 250, 250]      64
│    └─Conv2d: 2-3                       [1024, 16, 124, 124]      2,320
├─Block: 1-2                             [1024, 32, 61, 61]        --
│    └─Conv2d: 2-4                       [1024, 32, 124, 124]      4,640
│    └─Conv2d: 2-5                       [1024, 32, 124, 124]      544
│    └─Conv2d: 2-6                       [1024, 32, 61, 61]        9,248
├─Block: 1-3                             [1024, 64, 30, 30]        --
│    └─Conv2d: 2-7                       [1024, 64, 61, 61]        18,496
│    └─Conv2d: 2-8                       [1024, 64, 61, 61]        2,112
│    └─Conv2d: 2-9                       [1024, 64, 30, 30]        

In [36]:
images = []
tags = []

import cv2, os
for i in os.listdir("/content/forest_fire/Training and Validation"):
  print(i)
  5/0

nofire


ZeroDivisionError: ignored